In [5]:
from keras_ocr_tflite import build_model, CTCDecoder, model, DEFAULT_ALPHABET
import cv2
import numpy as np
import string

Looking for C:\Users\LAPTOP WORLD\.keras-ocr\crnn_kurapan.h5
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(1, 31, 200, 1)]    0           []                               
                                                                                                  
 permute (Permute)              (1, 200, 31, 1)      0           ['input[0][0]']                  
                                                                                                  
 lambda (Lambda)                (1, 200, 31, 1)      0           ['permute[0][0]']                
                                                                                                  
 conv_1 (Conv2D)                (1, 200, 31, 64)     640         ['lambda[0][0]']                 
                               

In [15]:
image = cv2.imread(r"C:\python\projects\Upwork-Projects\Sara RCNN\rego_images\data\data - 20230403_141503-KP369-a7748e94-915f-48b6-91d1-8a56f3641129-cutout.jpg", cv2.IMREAD_GRAYSCALE)
image = cv2.resize(image, (200, 31))
image = np.expand_dims(image, axis=-1)
image = np.expand_dims(image, axis=0)
image = image / 255.0  # Normalize image

prediction = model.predict(image)
decoded_text = CTCDecoder()(prediction)
text = ''.join([DEFAULT_ALPHABET[i] for i in decoded_text[0] if i != -1])

print(text)

1/1 [==============================] - 0s 125ms/step
kibgs


In [19]:
def preprocess(img):
    (h, w) = img.shape
    
    final_img = np.ones([64, 256])*255 # blank white image
    
    # crop
    if w > 256:
        img = img[:, :256]
        
    if h > 64:
        img = img[:64, :]
    
    
    final_img[:h, :w] = img
    return cv2.rotate(final_img, cv2.ROTATE_90_CLOCKWISE)

In [20]:
image = cv2.imread(r"C:\python\projects\Upwork-Projects\Sara RCNN\rego_images\data\data - 20230403_141503-KP369-a7748e94-915f-48b6-91d1-8a56f3641129-cutout.jpg", cv2.IMREAD_GRAYSCALE)
image = preprocess(image)
image = image / 255.  # Normalize the image
image = np.expand_dims(image, axis=0)  # Add batch dimension

In [26]:
import tensorflow as tf

interpreter = tf.lite.Interpreter(model_path='2.tflite')
interpreter.allocate_tensors()

output = interpreter.get_output_details()[0]  # Model has single output.
input = interpreter.get_input_details()[0]  # Model has single input.

# Set input tensor
interpreter.set_tensor(input['index'], image.astype(np.float32))

# Run the interpreter
interpreter.invoke()

# Get output tensor
output_data = interpreter.get_tensor(output[0]['index'])

ValueError: Cannot set tensor: Dimension mismatch. Got 3 but expected 4 for input 0.

In [24]:
input

{'name': 'serving_default_input:0',
 'index': 0,
 'shape': array([  1,  31, 200,   1]),
 'shape_signature': array([  1,  31, 200,   1]),
 'dtype': numpy.float32,
 'quantization': (0.0, 0),
 'quantization_parameters': {'scales': array([], dtype=float32),
  'zero_points': array([], dtype=int32),
  'quantized_dimension': 0},
 'sparsity_parameters': {}}

In [6]:
# Assuming you have an image stored in the variable "image"
image = cv2.imread(r"C:\python\projects\Upwork-Projects\Sara RCNN\rego_images\data\data - 20230403_141503-KP369-a7748e94-915f-48b6-91d1-8a56f3641129-cutout.jpg", cv2.IMREAD_ANYCOLOR)
image = preprocess(image)
image = image / 255.  # Normalize the image
image = np.expand_dims(image, axis=0)  # Add batch dimension
# Make a prediction using the loaded model
predictions = interpreter.predict(image)

AttributeError: 'Interpreter' object has no attribute 'predict'

In [28]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import requests
from PIL import Image

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")

# load image from the IAM dataset
url = "https://fki.tic.heia-fr.ch/static/img/a01-122-02.jpg"
image = Image.open(requests.get(url, stream=True).raw).convert("RGB")

pixel_values = processor(image, return_tensors="pt").pixel_values
generated_ids = model.generate(pixel_values)

generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

c:\Users\LAPTOP WORLD\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\LAPTOP WORLD\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Could not find image processor class in the image processor config or the model config. Loading based on p

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-handwritten and are newly initialized: ['encoder.pooler.dense.weight', 'encoder.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


c:\Users\LAPTOP WORLD\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\generation\utils.py:1346: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [10]:
import os
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import pandas as pd

images = []
pred_labels = []
data_path = r'C:\python\projects\Upwork-Projects\Sara RCNN\rego_images\data'
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")

for img in os.listdir(data_path):

    images.append(img)

    image = Image.open(os.path.join(data_path,img)).convert("RGB")

    pixel_values = processor(image, return_tensors="pt").pixel_values
    generated_ids = model.generate(pixel_values)

    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

    pred_labels.append(generated_text)

    print('-',end='')

df = pd.DataFrame({'images': images, 'predicted label': pred_labels})
df.head()

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\LAPTOP WORLD\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\generation\utils.py:1346: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

,images,predicted label
0,data - 20230403_094217-6172f02f-75df-4e7c-b1dd...,turners.
1,data - 20230403_094217-96b2f11d-9f00-434c-a838...,votes.
2,data - 20230403_094217-a33648aa-fda3-40e9-bdf1...,tombers.
3,data - 20230403_094217-ae807121-7d6a-43b2-8193...,to raise
4,data - 20230403_094217-bbb7f34d-860e-4d1a-aa1d...,outcome


In [11]:
df.to_csv('prediction.csv', index=False)

In [19]:
from huggingface_hub import from_pretrained_keras
import tensorflow as tf

model = from_pretrained_keras("keras-io/ocr-for-captcha")


for img in os.listdir(data_path):

    images.append(img)

    # 1. Read image
    img = tf.io.read_file(os.path.join(data_path,img))
    # 2. Decode and convert to grayscale
    img = tf.io.decode_png(img, channels=1)
    # 3. Convert to float32 in [0, 1] range
    img = tf.image.convert_image_dtype(img, tf.float32)
    # 4. Resize to the desired size
    img = tf.image.resize(img, [50, 200])
    # 5. Transpose the image because we want the time
    # dimension to correspond to the width of the image.
    img = tf.transpose(img, perm=[1, 0, 2])

    generated_text = model.predict(img)

    pred_labels.append(generated_text)

    print('-',end='')

df = pd.DataFrame({'images': images, 'predicted label': pred_labels})
df.head()

config.json not found in HuggingFace Hub.


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

ValueError: in user code:

    File "c:\Users\LAPTOP WORLD\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\LAPTOP WORLD\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\LAPTOP WORLD\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\LAPTOP WORLD\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1983, in predict_step
        return self(x, training=False)
    File "c:\Users\LAPTOP WORLD\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb

In [13]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
# import requests
from PIL import Image

processor = TrOCRProcessor.from_pretrained("keras-io/ocr-for-captcha")
model = VisionEncoderDecoderModel.from_pretrained("keras-io/ocr-for-captcha")

# load image from the IAM dataset




image = Image.open(r"C:\python\projects\Upwork-Projects\Sara RCNN\rego_images\data\data - 20230403_141503-KP369-a7748e94-915f-48b6-91d1-8a56f3641129-cutout.jpg").convert("RGB")

pixel_values = processor(image, return_tensors="pt").pixel_values
generated_ids = model.generate(pixel_values)

generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

OSError: keras-io/ocr-for-captcha does not appear to have a file named preprocessor_config.json. Checkout 'https://huggingface.co/keras-io/ocr-for-captcha/main' for available files.

In [5]:
generated_text

'KP369'

In [30]:
from transformers import (
    TrOCRConfig,
    TrOCRProcessor,
    TrOCRForCausalLM,
    ViTConfig,
    ViTModel,
    VisionEncoderDecoderModel,
)
import requests
from PIL import Image

# TrOCR is a decoder model and should be used within a VisionEncoderDecoderModel
# init vision2text model with random weights
encoder = ViTModel(ViTConfig())
decoder = TrOCRForCausalLM(TrOCRConfig())
model = VisionEncoderDecoderModel(encoder=encoder, decoder=decoder)

# If you want to start from the pretrained model, load the checkpoint with `VisionEncoderDecoderModel`
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")

# load image from the IAM dataset
url = r"C:\python\projects\Upwork-Projects\Sara RCNN\rego_images\data\data - 20230403_141503-KP369-a7748e94-915f-48b6-91d1-8a56f3641129-cutout.jpg"
image = Image.open(requests.get(url, stream=True).raw).convert("RGB")
pixel_values = processor(image, return_tensors="pt").pixel_values
text = "industry, ' Mr. Brown commented icily. ' Let us have a"

# training
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
model.config.vocab_size = model.config.decoder.vocab_size

labels = processor.tokenizer(text, return_tensors="pt").input_ids
outputs = model(pixel_values, labels=labels)
loss = outputs.loss
round(loss.item(), 2)

# inference
generated_ids = model.generate(pixel_values)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
generated_text

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-handwritten and are newly initialized: ['encoder.pooler.dense.weight', 'encoder.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


InvalidSchema: No connection adapters were found for 'C:\\python\\projects\\Upwork-Projects\\Sara RCNN\\rego_images\\data\\data - 20230403_141503-KP369-a7748e94-915f-48b6-91d1-8a56f3641129-cutout.jpg'

In [12]:
import cv2 
import pytesseract
import os
import pandas as pd

images = []
pred_labels = []
data_path = r'C:\python\projects\Upwork-Projects\Sara RCNN\rego_images\dataS'
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

for img in os.listdir(data_path):

    images.append(img)

    img = cv2.imread(os.path.join(data_path,img))


    # Adding custom options
    custom_config = r'--oem 3 --psm 6'

    pred_labels.append(pytesseract.image_to_string(img, config=custom_config).replace('\n',''))

    print('-',end='')

df = pd.DataFrame({'images': images, 'predicted label': pred_labels})
df.head()

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

,images,predicted label
0,data - 20230403_094217-6172f02f-75df-4e7c-b1dd...,GTe6s-
1,data - 20230403_094217-96b2f11d-9f00-434c-a838...,WW 8o5
2,data - 20230403_094217-a33648aa-fda3-40e9-bdf1...,GTéEZ
3,data - 20230403_094217-ae807121-7d6a-43b2-8193...,Gia63
4,data - 20230403_094217-bbb7f34d-860e-4d1a-aa1d...,Qi a6ez_


In [11]:
df.to_csv('predictionSS.csv', index=False)